In [6]:
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import DataFrame, Series
import re # regular expressions
#adding graphics
import matplotlib.pyplot as plt
%pylab inline
from pylab import rcParams
from collections import defaultdict
from collections import OrderedDict as od
rcParams['figure.figsize'] = 18, 6

Populating the interactive namespace from numpy and matplotlib


In [7]:
#all_df = pd.read_table('balances.csv', sep=',',index_col='Ref_Date') 
all_lic = pd.read_table('business.licences.csv', sep=',') 
print all_lic
sub_lic = all_lic[["Issued","Cancel Date","Operating Name","Licence Address Line 3",'Licence No.','Category']]
print sub_lic.shape
null_Issued= sub_lic[pd.isnull(all_lic["Issued"])]
print null_Issued.shape[0]
null_name= sub_lic[pd.isnull(all_lic["Operating Name"])]
print null_name.shape[0]
null_postal= sub_lic[pd.isnull(all_lic["Licence Address Line 3"])]
print null_postal.shape[0]

                                       Category Licence No.  \
0                                TAXICAB BROKER  B032020976   
1                                TAXICAB BROKER  B032025475   
2                                TAXICAB BROKER  B032022604   
3                                TAXICAB BROKER  B030000236   
4                                TAXICAB BROKER  B032018529   
5                                TAXICAB BROKER  B032020423   
6                                TAXICAB BROKER  B032022904   
7                                TAXICAB BROKER  B030008288   
8                                TAXICAB BROKER  B033010926   
9                                TAXICAB BROKER  B030000300   
10                               TAXICAB BROKER  B030000301   
11                               TAXICAB BROKER  B030000296   
12                               TAXICAB BROKER  B033009456   
13                               TAXICAB BROKER  B033087407   
14                               TAXICAB BROKER  B03201

In [8]:
good_postal = sub_lic[pd.notnull(all_lic['Licence Address Line 3'])]
print good_postal.shape[0]

def getpostal(postal) :
    return postal[:3]+ postal[4:]


good_postal['postal']=good_postal['Licence Address Line 3'].map(getpostal)

print good_postal

144462
          Issued Cancel Date                    Operating Name  \
0       24/03/95    01/05/97                      EMERALD TAXI   
1       24/06/96    20/05/97                      PREMIER TAXI   
2       17/08/95    31/12/97                    A TORONTO TAXI   
3       03/03/88    08/06/98                       KIPLING KAB   
4       14/07/94    08/06/98                      RAINBOW TAXI   
5       02/02/95    24/07/98                        CROWN TAXI   
6       20/09/95    31/12/98                                     
7       25/04/80    04/01/99              EAST-END/HOWARD TAXI   
8       23/10/97    12/01/99                        RAPID TAXI   
9       28/03/94    23/03/99                               NaN   
10      28/03/94    23/03/99                               NaN   
11      17/01/92    04/05/99                     BEE LINE TAXI   
12      22/08/97    07/07/99                         MEGA-CITY   
13      24/07/98    18/08/99                        CROWN TAXI   
14 

/Users/ahmed/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
# in this part we read the ontario csv to lookup coordinates
onp = pd.read_table('ontario.csv', sep=',') 
print onp.shape

match3= lambda x : x[:3]
def getpostalcoordinates (code):
    if len(code)==3 :
         postal = onp[onp['postal'].map(match3)==code]
         
    if (len(code)==6):
         postal = onp[onp['postal'] ==code]
    if len(postal.index) > 1:
        postal = postal[:1]
    return postal

def coord(x) :
    c = getpostalcoordinates(x)
    if c is None :
        return 0,0
    if c.empty:
        return 0,0
    return c.iloc[0]['lat'],c.iloc[0]['lng']

# here we build a list of all zip codes in a list
def load_coord(dataset) :
    target = dataset
    target['lat'],target['lng']= zip(*target['postal'].map(coord))
    return target

coordinates = load_coord(good_postal)
coordinates.to_csv('ready_data.csv')
print coordinates

/Users/ahmed/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/ahmed/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(313072, 7)
          Issued Cancel Date                    Operating Name  \
0       24/03/95    01/05/97                      EMERALD TAXI   
1       24/06/96    20/05/97                      PREMIER TAXI   
2       17/08/95    31/12/97                    A TORONTO TAXI   
3       03/03/88    08/06/98                       KIPLING KAB   
4       14/07/94    08/06/98                      RAINBOW TAXI   
5       02/02/95    24/07/98                        CROWN TAXI   
6       20/09/95    31/12/98                                     
7       25/04/80    04/01/99              EAST-END/HOWARD TAXI   
8       23/10/97    12/01/99                        RAPID TAXI   
9       28/03/94    23/03/99                               NaN   
10      28/03/94    23/03/99                               NaN   
11      17/01/92    04/05/99                     BEE LINE TAXI   
12      22/08/97    07/07/99                         MEGA-CITY   
13      24/07/98    18/08/99                        CROWN TAXI  

In [94]:
badpostals=coordinates[coordinates['lat']==0]
print badpostals.shape

(290, 8)


In [99]:
badpostals.to_csv('bad_postals.csv')
line= badpostals[badpostals['Licence No.']=='B500025967'].iloc[0]['Licence Address Line 3']
print line
print "-"+line+'-'
sp = badpostals[badpostals['Licence Address Line 3']=='       ']
print sp.shape

l = lambda x:len(x)
badpostals["len"]=badpostals['Licence Address Line 3'].map(l)
print badpostals[badpostals['len']==3].shape
print badpostals[badpostals['len']>3].shape


       
-       -
(18, 9)
(125, 9)
(165, 9)


/Users/ahmed/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
coordinates.to_csv('bus_lic_munged.csv')

        Unnamed: 0  postal        lat        lng     city prov Unnamed: 5
177578      519336  M2H0A1  43.801874 -79.344606  TORONTO   ON        NaN
        Unnamed: 0  postal        lat        lng       city prov Unnamed: 5
190889      532649  M4J4J7  43.684284 -79.331017  East York   ON        NaN
